# Laboratorium 6 - rekomendacje grupowe

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab6`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas matplotlib`

## Część 1. - przygotowanie danych

In [6]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from random import choice, sample
from statistics import mean, stdev

from reco_utils import *

In [7]:
# wczytujemy oceny uytkownikow i obliczamy (za pomocą collaborative filtering) wszystkie przewidywane oceny filmow

raw_ratings = pandas.read_csv('ratings.csv').drop(columns=['timestamp'])
movies = list(raw_ratings['movieId'].unique())
users = list(raw_ratings['userId'].unique())
ratings = get_predicted_ratings(raw_ratings)
ratings

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,10,8,9,7,9,8,9,8,9,8,7,9,8,8,8,10,6,9,9,8,8,9,9,9,8,9,7,8,8,7,8,9,9,8,6,10,8,8,8,7,...,9,9,7,10,8,8,9,8,8,9,9,7,7,9,9,6,8,8,8,8,7,10,7,8,7,9,9,7,8,7,8,7,7,9,8,9,8,8,8,7
2,5,5,5,4,5,4,5,4,5,5,4,5,5,5,5,5,4,5,5,4,5,5,5,5,5,5,4,5,4,4,4,6,5,5,3,6,4,4,5,4,...,5,5,4,5,5,4,5,5,5,5,5,4,4,5,5,4,5,5,5,5,4,5,4,5,4,5,5,4,5,4,5,3,4,5,5,5,5,4,4,4
3,5,4,4,3,4,4,4,4,4,4,3,4,4,4,4,5,3,4,4,4,4,4,4,4,4,5,4,4,4,3,4,4,5,4,3,5,4,4,4,3,...,4,4,4,4,4,4,4,4,4,4,4,3,3,4,4,3,4,4,4,4,3,5,3,4,4,4,4,3,4,3,4,3,4,4,4,5,4,3,3,4
4,8,7,7,6,7,6,8,6,7,7,6,7,7,7,7,7,5,7,8,7,7,8,7,7,7,7,6,7,7,6,6,8,8,7,5,8,6,6,7,6,...,7,7,6,8,7,6,8,7,7,7,8,6,6,7,7,5,7,6,6,7,6,8,6,7,6,8,7,6,7,5,6,6,6,7,7,8,6,6,6,6
5,5,5,5,4,5,4,5,4,5,5,4,5,5,5,4,5,4,5,5,5,5,5,5,5,5,5,4,5,4,4,4,6,6,5,3,6,4,4,5,4,...,5,5,4,5,4,4,5,5,5,5,6,4,4,5,5,4,5,4,4,5,4,6,4,5,5,5,5,4,4,4,5,4,4,5,5,5,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,8,7,8,6,8,7,8,7,8,7,6,8,7,7,7,8,5,8,8,7,8,8,8,8,7,8,7,7,7,7,7,9,9,7,5,9,7,6,7,6,...,8,8,6,8,7,7,8,7,7,8,8,6,6,7,8,6,7,7,7,7,6,9,6,7,7,8,7,7,7,6,7,6,6,8,7,8,7,7,6,6
607,8,7,8,6,8,7,8,7,8,7,6,8,7,7,7,8,5,8,8,7,8,8,7,7,7,8,6,7,7,7,7,9,8,7,5,9,7,7,7,6,...,8,8,6,8,7,7,8,7,7,8,8,6,6,7,7,6,7,7,7,7,6,9,6,7,7,8,7,6,7,6,7,6,6,7,7,8,7,7,7,6
608,8,6,7,5,7,6,7,6,6,6,5,7,7,6,6,7,5,7,7,6,7,7,7,6,6,7,5,6,6,5,6,7,7,6,4,8,6,6,6,5,...,6,7,5,7,6,6,7,6,7,6,7,5,5,6,6,5,6,6,6,6,5,8,5,6,5,7,6,5,6,5,6,5,6,6,6,7,6,6,5,5
609,5,4,5,4,5,4,4,4,5,4,4,5,4,4,4,5,3,5,5,4,4,5,5,5,5,5,4,4,4,4,4,5,5,4,3,5,4,4,4,4,...,5,5,4,5,4,4,5,4,4,5,5,4,4,5,5,3,5,4,4,4,3,5,3,4,4,4,4,4,4,3,4,4,4,5,4,5,5,4,4,3


In [8]:
# definiujemy testowe grupy uzytkownikow, dla ktorych bedziemy generowac rekomendacje

groups_no = 50
group_size = 5
groups = [sample(users, group_size) for i in range(groups_no)]
groups

[[55, 204, 248, 88, 138],
 [339, 290, 166, 523, 462],
 [334, 14, 240, 162, 16],
 [68, 54, 224, 412, 549],
 [576, 21, 571, 358, 200],
 [388, 527, 185, 544, 150],
 [336, 487, 477, 519, 586],
 [539, 154, 39, 567, 442],
 [449, 274, 15, 154, 243],
 [507, 176, 44, 77, 24],
 [470, 276, 207, 160, 181],
 [465, 96, 151, 260, 193],
 [277, 104, 565, 15, 71],
 [307, 309, 72, 2, 605],
 [293, 311, 359, 133, 20],
 [359, 126, 571, 580, 105],
 [598, 463, 188, 434, 205],
 [463, 42, 110, 115, 487],
 [602, 328, 391, 73, 427],
 [604, 530, 129, 147, 210],
 [512, 557, 513, 256, 64],
 [42, 428, 403, 172, 281],
 [51, 296, 428, 488, 240],
 [9, 560, 562, 261, 540],
 [214, 60, 366, 27, 487],
 [451, 587, 24, 192, 463],
 [229, 313, 98, 575, 559],
 [99, 394, 271, 204, 281],
 [539, 381, 394, 218, 326],
 [159, 67, 497, 539, 265],
 [543, 480, 554, 401, 513],
 [25, 390, 283, 317, 410],
 [241, 126, 425, 206, 192],
 [334, 112, 351, 66, 212],
 [256, 269, 261, 241, 7],
 [607, 61, 566, 424, 604],
 [299, 465, 570, 99, 101],
 [

## Część 2. - algorytmy proste

In [9]:
# zdefiniujmy interfejs dla wszystkich algorytmow rekomendacyjnych

class Recommender:
    def recommend(self, movies, ratings, group, size):
        pass


# jako pierwszy zaimplementujemy algorytm losowy - dla porownania
    
class RandomRecommender(Recommender):
    def __init__(self):
        self.name = 'random'
        
    def recommend(self, movies, ratings, group, size):
        return sample(movies, size)

In [10]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen

class AverageRecommender(Recommender):
    def __init__(self):
        self.name = 'average'
    
    def recommend(self, movies, ratings, group, size):
        return sorted(movies, key=lambda movie: self.score(movie, ratings, group), reverse=True)[:size]
        
    def score(self, movie_id, ratings, group):
        average =  1.0 * sum([ratings[movie_id][user] for user in group]) / len(group)
        return average

In [11]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen,
#   ale rownoczesnie wykluczajacy te filmy, ktore otrzymaly choc jedna ocene ponizej thresholdu

class AverageWithoutMiseryRecommender(Recommender):
    def __init__(self, score_threshold):
        self.name = 'average_without_misery'
        self.score_threshold = score_threshold
        
    def recommend(self, movies, ratings, group, size):
        movies_without_misery = [m for m in movies if self.min_rating(m, ratings, group) > self.score_threshold]
        return sorted(movies, key=lambda movie: self.score(movie, ratings, group), reverse=True)[:size]
        
    def score(self, movie_id, ratings, group):
        average =  1.0 * sum([ratings[movie_id][user] for user in group]) / len(group)
        return average
    
    def min_rating(self, movie_id, ratings, group):
        return min([ratings[movie_id][user] for user in group])

In [12]:
# algorytm uwzgledniajacy preferencje tylko jednego uzytkownika

class DictatorshipRecommender(Recommender):
    def __init__(self, dictator_id):
        self.name = 'dictatorship'
        self.dictator_id = dictator_id
        
    def recommend(self, movies, ratings, group, size):
        return sorted(movies, key=lambda m: ratings[m][self.dictator_id], reverse=True)[:size]

In [21]:
# algorytm, ktory w kazdej turze uwzglednia preferencje tylko jednego, kolejnego uzytkownika
from random import choice


class FairnessRecommender(Recommender):
    def __init__(self):
        self.name = 'fairness'
    
    def recommend(self, movies, ratings, group, size):
        iteration, user_nr = 0, 0
        recommendation = []
        while iteration < size:
          
          next_movie = choice(sorted(movies, key = lambda m: ratings[m][group[user_nr]], reverse=True)[:10])
          recommendation.append(next_movie)

          user_nr += 1
          if user_nr >= len(group): user_nr = 0
          iteration += 1
        return recommendation

FairnessRecommender().recommend(movies, ratings, groups[0], 10)

[1136, 593, 117887, 2046, 142550, 2046, 260, 32657, 6350, 142550]

In [30]:
# algorytm zachlanny, aproksymujacy metode Proportional Approval Voting
#   w kazdej iteracji wybieramy ten film, ktory najbardziej zwieksza zadowolenie zgodnie z punktacja PAV

class PAVRecommender(Recommender):
    def __init__(self):
        self.name = 'PAV'
        self.threshold = 7
        self.PAV = None
        
    def recommend(self, movies, ratings, group, size):
        self.PAV = {user: 0 for user in group}
        top_movies = []
        movies_copy = movies.copy()
        for _ in range(size):
          movie_to_recommend = sorted(movies_copy, key = lambda x: self.get_PAV(group, ratings[x]), reverse=True)[0]
          for user in group:
            if ratings[movie_to_recommend][user] > self.threshold:
              self.PAV[user] += 1
          top_movies.append(movie_to_recommend)
          movies_copy.remove(movie_to_recommend)
        return top_movies

    def get_PAV(self, group, ratings_mov):
        score = 0
        for user in group:
          if ratings_mov[user] > self.threshold:
            score += (1.0 / (self.PAV[user]+1))
        return score

PAVRecommender().recommend(movies, ratings, groups[0], 10)

[32657, 6460, 117887, 114494, 1136, 2046, 318, 265, 6350, 126548]

## Część 3. - funkcje celu

In [33]:
# dwie funkcje pomocnicze:
#  - znajdujaca ulubione filmy danego uzytkownika
#  - obliczajaca sume ocen wystawionych przez uzytkownika wszystkim filmom w rekomendacji

def top_n_movies_for_user(ratings, movies, user_id, n):
    return sorted(movies, key=lambda m: ratings[m][user_id], reverse=True)[:n]

def total_score(recommendation, user_id, ratings):
    return sum(ratings[movie][user_id] for movie in recommendation)

total_score([1,2,3], 10, ratings)

20

In [37]:
# funkcja obliczajaca zadowolenie pojedynczego uzytkownika
#  - iloraz zadowolenia z wygenerowanej rekomendacji oraz zadowolenia z hipotetycznej rekomendacji idealnej
def overall_user_satisfaction(recommendation, user_id, movies, ratings):
    # ... total_score/score_usera
    return total_score(top_n_movies_for_user(ratings, movies, user_id, len(recommendation)), user_id, ratings) / total_score(recommendation, user_id, ratings)
print(overall_user_satisfaction([1,2,3], 10, movies, ratings))

# funkcja celu - srednia z zadowolenia wszystkich uzytkownikow w grupie
def overall_group_satisfaction(recommendation, group, movies, ratings):
    return 1.0 * sum([overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group]) / len(group)

# funkcja celu - roznica miedzy maksymalnym i minimalnym zadowolenie w grupie
def group_dissatisfaction(recommendation, group, movies, ratings):
    satisfaction_scores = [overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group]
    return max(satisfaction_scores) - min(satisfaction_scores)

1.35


## Część 4. - Sequential Hybrid Aggregation

In [42]:
# algorytm balansujacy pomiedzy wyborem elementow o najwyzszej sredniej ocen
#   i o najwyzszej minimalnej ocenie
#   wyliczajacy w kazdej iteracji parametr alfa - jak na wykladzie

def avg(arr):
    return 1.0 * sum(arr) / len(arr)

class SequentialHybridAggregationRecommender(Recommender):
      def __init__(self):
        self.name = 'sequential_hybrid_aggregation'
        self.alpha = 0.5

      def get_score(self, ratings, movie, group):
        score = (1.0 - self.alpha) * avg(ratings[movie]) + self.alpha * min(ratings[movie])  
        self.update_alpha(group, movie, ratings)
        return score

      def update_alpha(self, group, movie, ratings):
        self.alpha = (max([ratings[movie][user] for user in group]) - min([ratings[movie][user] for user in group])) / 10

      def recommend(self, movies, ratings, group, size):
        return sorted(movies, key = lambda m: self.get_score(ratings, m, group))[:size]
    
SequentialHybridAggregationRecommender().recommend(movies, ratings, groups[0], 10)

[5205, 60647, 103027, 138546, 115727, 27473, 1516, 7235, 134184, 170551]

## Część 5. - porównanie algorytmów

In [49]:
recommenders = [
    RandomRecommender(),
    AverageRecommender(),
    AverageWithoutMiseryRecommender(2),
    DictatorshipRecommender(1),
    FairnessRecommender(),
    PAVRecommender(),
    SequentialHybridAggregationRecommender()
]

recommendation_size = 10

# dla kazdego algorytmu:
#  - wygenerujmy jedna rekomendacje dla kazdej grupy
#  - obliczmy wartosci funkcji celu dla kazdej rekomendacji
#  - obliczmy srednia i odchylenie standardowe wartosci funkcji celu
#  - wypiszmy wyniki na konsole

data = {recommender.name: {"mean": None, "stdev": None} for recommender in recommenders}

for recommender in recommenders:
  satisfactions_for_recommender = []
  for group in groups:
    recommendation = recommender.recommend(movies, ratings, group, recommendation_size)
    satisfactions_for_recommender.append(overall_group_satisfaction(recommendation, group, movies, ratings))
  data[recommender.name]["mean"] = mean(satisfactions_for_recommender)
  data[recommender.name]["stdev"] = stdev(satisfactions_for_recommender)

from pprint import pprint as pp

pp(data) 

{'PAV': {'mean': 1.0792475125302217, 'stdev': 0.05626219504804478},
 'average': {'mean': 1.0043531925609313, 'stdev': 0.005605994091191699},
 'average_without_misery': {'mean': 1.0043531925609313,
                            'stdev': 0.005605994091191699},
 'dictatorship': {'mean': 1.129063878855781, 'stdev': 0.018112331611441566},
 'fairness': {'mean': 1.0426510537775047, 'stdev': 0.017595314236797208},
 'random': {'mean': 1.4225458418615209, 'stdev': 0.06182549526625682},
 'sequential_hybrid_aggregation': {'mean': 2.3526127387412985,
                                   'stdev': 0.06433094087545993}}


In [50]:
sorted([(recommender.name, data[recommender.name]["mean"]) for recommender in recommenders], key = lambda x: x[1], reverse = True)

[('sequential_hybrid_aggregation', 2.3526127387412985),
 ('random', 1.4225458418615209),
 ('dictatorship', 1.129063878855781),
 ('PAV', 1.0792475125302217),
 ('fairness', 1.0426510537775047),
 ('average', 1.0043531925609313),
 ('average_without_misery', 1.0043531925609313)]

In [52]:
sorted([(recommender.name, data[recommender.name]["stdev"]) for recommender in recommenders], key = lambda x: x[1], reverse = True)

[('sequential_hybrid_aggregation', 0.06433094087545993),
 ('random', 0.06182549526625682),
 ('PAV', 0.05626219504804478),
 ('dictatorship', 0.018112331611441566),
 ('fairness', 0.017595314236797208),
 ('average', 0.005605994091191699),
 ('average_without_misery', 0.005605994091191699)]